In [1]:
import streamlit as st
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_core.runnables import RunnableLambda
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter
from config_env import embedding_endpoint, llm_endpoint, AZURE_OPENAI_VERSION


In [2]:
# Initialize models and vector store
api_key = 'd7fd42addeff4f4a91f9beea8996f4cc'  # Replace with your actual API key

embedding_model = AzureOpenAIEmbeddings(azure_endpoint=embedding_endpoint, api_key=api_key)
vector_store = InMemoryVectorStore(embedding_model).load('vector_store_v1', embedding_model)
open_ai_llm = AzureChatOpenAI(
    openai_api_version=AZURE_OPENAI_VERSION,
    azure_endpoint=llm_endpoint,
    temperature=0,
    api_key=api_key
)

In [5]:
response_prompt = ChatPromptTemplate.from_template("""
Instructions: You are a customer service assistant chatbot at Bank Mandiri known as Mita. Your main role is to assist customers by providing accurate information, answering questions, and resolving issues related to our products and services. 
Always use bahasa indonesia in response.
You are an assistant for answering specific questions related to Bank Mandiri.
If a question is outside the topic of Bank Mandiri and our products or services, kindly state that the question is not relevant. If a user simply wants to try the service or asks who you are, introduce yourself and ask how you can assist them. 
Reformat your answers to be more straigtforward
Main Guidelines:
1. Polite and Professional Tone: Always communicate in a friendly and professional manner.
2. Empathy and Understanding: Acknowledge customer concerns and express understanding.
3. Clarity and Accuracy: Provide clear, concise, and accurate information.
4. Focus on Problem Resolution: Strive to resolve issues efficiently and effectively.
5. Escalation Protocol: If a customer's issue cannot be resolved, inform them that their question will be escalated to a human representative.
                                                   
You are designed to learn from interactions, so continuously improve your responses based on customer feedback.
Context: {context}

Question: {question}
""")

In [6]:
def create_hypothetical_rag_chain(vectorstore):
    def retrieve_docs(query: str) -> str:
        docs = vectorstore.similarity_search(query, k=3)
        return summarize_content("\n\n".join(doc.page_content for doc in docs))

    retriever = RunnableLambda(retrieve_docs)

    chain = (
        {
            "question": itemgetter("original_question"),
            "context": lambda x: retriever.invoke(x["question"]),
        }
        | response_prompt
        | open_ai_llm
    )
    
    return chain

In [7]:
def summarize_content(content):
    summary_prompt = ChatPromptTemplate.from_template("""
    Instruction: "gabungkan semua informasi yang didapat dan kelompokkan
    tampilkan informasi paling relevan" 
    {content}
    """)
    
    summary_chain = summary_prompt | open_ai_llm
    summary_response = summary_chain.invoke({"content": content})
    return summary_response.content


In [9]:
RESPONSES = {
    'id': {
        'greeting': "👋 Hai! Saya Mita, asisten layanan pelanggan Bank Mandiri. Untuk memberikan pelayanan terbaik silahkan bertanya atau beri saya instruksi",
        'short_greeting': "👋 Hai! Saya Mita, asisten layanan pelanggan Bank Mandiri.",
        'language_prompt': "Untuk memberikan pelayanan terbaik, apakah Anda lebih nyaman menggunakan Bahasa Indonesia atau English?",
        'irrelevant': "Maaf, pertanyaan tersebut tidak relevan dengan produk atau layanan Bank Mandiri. Silakan ajukan pertanyaan lain seputar layanan kami!",
        'error': "Maaf, terjadi kesalahan dalam memproses permintaan Anda. Silakan coba lagi.",
        'clarification': "Mohon maaf, saya kurang memahami maksud Anda. Bisakah Anda menjelaskan lebih detail?",
    },
    'en': {
        'greeting': "👋 Hi! I'm Mita, Bank Mandiri's customer service assistant. To provide the best service, please give me instruction or ask anything",
        'short_greeting': "👋 Hi! I'm Mita, Bank Mandiri's customer service assistant.",
        'language_prompt': "To provide the best service, would you prefer to communicate in Bahasa Indonesia or English?",
        'irrelevant': "I apologize, but that question isn't relevant to Bank Mandiri's products or services. Please feel free to ask about our services!",
        'error': "I apologize, but there was an error processing your request. Please try again.",
        'clarification': "I'm sorry, I didn't quite understand. Could you please elaborate?",
    }
}

In [10]:
def detect_language_preference(text):
    """Simple language detection based on common words"""
    # For simplicity, we're always returning 'id' here. In a real implementation,
    # you'd want to implement actual language detection logic.
    return 'id'

def get_response(key, lang='id'):
    """Get response in the specified language"""
    return RESPONSES[lang][key]


In [13]:
import streamlit as st
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_core.runnables import RunnableLambda
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter
from config_env import embedding_endpoint, llm_endpoint, AZURE_OPENAI_VERSION

st.set_page_config(page_title="Hypothetical RAG Chatbot", layout="wide")

# Initialize models and vector store
api_key = 'd7fd42addeff4f4a91f9beea8996f4cc'  # Replace with your actual API key

embedding_model = AzureOpenAIEmbeddings(azure_endpoint=embedding_endpoint, api_key=api_key)
vector_store = InMemoryVectorStore(embedding_model).load('vector_store_v1', embedding_model)
open_ai_llm = AzureChatOpenAI(
    openai_api_version=AZURE_OPENAI_VERSION,
    azure_endpoint=llm_endpoint,
    temperature=0,
    api_key=api_key
)

# Chat prompt template
response_prompt = ChatPromptTemplate.from_template("""
Instructions: You are a customer service assistant chatbot at Bank Mandiri known as Mita. Your main role is to assist customers by providing accurate information, answering questions, and resolving issues related to our products and services. 
Always use bahasa indonesia in response.
You are an assistant for answering specific questions related to Bank Mandiri.
If a question is outside the topic of Bank Mandiri and our products or services, kindly state that the question is not relevant. If a user simply wants to try the service or asks who you are, introduce yourself and ask how you can assist them. 
Reformat your answers to be more straigtforward
Main Guidelines:
1. Polite and Professional Tone: Always communicate in a friendly and professional manner.
2. Empathy and Understanding: Acknowledge customer concerns and express understanding.
3. Clarity and Accuracy: Provide clear, concise, and accurate information.
4. Focus on Problem Resolution: Strive to resolve issues efficiently and effectively.
5. Escalation Protocol: If a customer's issue cannot be resolved, inform them that their question will be escalated to a human representative.
                                                   
You are designed to learn from interactions, so continuously improve your responses based on customer feedback.
Context: {context}

Question: {question}
""")


def create_hypothetical_rag_chain(vectorstore):
    def retrieve_docs(query: str) -> str:
        docs = vectorstore.similarity_search(query, k=3)
        return summarize_content("\n\n".join(doc.page_content for doc in docs))

    retriever = RunnableLambda(retrieve_docs)

    chain = (
        {
            "question": itemgetter("original_question"),
            "context": lambda x: retriever.invoke(x["question"]),
        }
        | response_prompt
        | open_ai_llm
    )
    
    return chain

def summarize_content(content):
    summary_prompt = ChatPromptTemplate.from_template("""
    Instruction: "gabungkan semua informasi yang didapat dan kelompokkan
    tampilkan informasi paling relevan" 
    {content}
    """)
    
    summary_chain = summary_prompt | open_ai_llm
    summary_response = summary_chain.invoke({"content": content})
    return summary_response.content

# Predefined responses
RESPONSES = {
    'id': {
        'greeting': "👋 Hai! Saya Mita, asisten layanan pelanggan Bank Mandiri. Untuk memberikan pelayanan terbaik silahkan bertanya atau beri saya instruksi",
        'short_greeting': "👋 Hai! Saya Mita, asisten layanan pelanggan Bank Mandiri.",
        'language_prompt': "Untuk memberikan pelayanan terbaik, apakah Anda lebih nyaman menggunakan Bahasa Indonesia atau English?",
        'irrelevant': "Maaf, pertanyaan tersebut tidak relevan dengan produk atau layanan Bank Mandiri. Silakan ajukan pertanyaan lain seputar layanan kami!",
        'error': "Maaf, terjadi kesalahan dalam memproses permintaan Anda. Silakan coba lagi.",
        'clarification': "Mohon maaf, saya kurang memahami maksud Anda. Bisakah Anda menjelaskan lebih detail?",
    },
    'en': {
        'greeting': "👋 Hi! I'm Mita, Bank Mandiri's customer service assistant. To provide the best service, please give me instruction or ask anything",
        'short_greeting': "👋 Hi! I'm Mita, Bank Mandiri's customer service assistant.",
        'language_prompt': "To provide the best service, would you prefer to communicate in Bahasa Indonesia or English?",
        'irrelevant': "I apologize, but that question isn't relevant to Bank Mandiri's products or services. Please feel free to ask about our services!",
        'error': "I apologize, but there was an error processing your request. Please try again.",
        'clarification': "I'm sorry, I didn't quite understand. Could you please elaborate?",
    }
}

def detect_language_preference(text):
    """Simple language detection based on common words"""
    # For simplicity, we're always returning 'id' here. In a real implementation,
    # you'd want to implement actual language detection logic.
    return 'id'

def get_response(key, lang='id'):
    """Get response in the specified language"""
    return RESPONSES[lang][key]

# Initialize session state
if "messages" not in st.session_state:
    st.session_state.messages = []
if "language_set" not in st.session_state:
    st.session_state.language_set = False
if "selected_language" not in st.session_state:
    st.session_state.selected_language = None

# Initialize the RAG chain
rag_chain = create_hypothetical_rag_chain(vector_store)

# Streamlit UI
st.title("Your Truly Livin Partner")

# Display chat history
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

# User input handling
if prompt := st.chat_input("What would you like to know?"):
    st.session_state.messages.append({"role": "user", "content": prompt})
    
    with st.chat_message("user"):
        st.markdown(prompt)

    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            try:
                if not st.session_state.language_set:
                    st.session_state.selected_language = detect_language_preference(prompt)
                    st.session_state.language_set = True
                
                lang = st.session_state.selected_language
                
                response = rag_chain.invoke({
                    "question": prompt,
                    "original_question": prompt
                })

                full_response = response.content

                if "tidak relevan" in full_response.lower() or "not relevant" in full_response.lower():
                    st.markdown(get_response('irrelevant', lang))
                else:
                    st.markdown(full_response)

                st.session_state.messages.append({
                    "role": "assistant", 
                    "content": full_response
                })

            except Exception as e:
                st.error(get_response('error', lang))
                print(f"Error: {str(e)}")  # For debugging

2024-10-09 20:33:43.111 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-09 20:33:49.792 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-09 20:33:49.793 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-09 20:33:49.794 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-09 20:33:49.850 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-09 20:33:49.850 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-09 20:33:49.851 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-09 20:33:49.852 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar